In [1]:
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so

from pymutspec.draw import plot_mutspec12
from pymutspec.annotation import CodonAnnotation
from pymutspec.constants import possible_codons
from pymutspec.io import read_genbank_ref

from scipy.stats import chisquare, ks_2samp, pearsonr, spearmanr, uniform
from scipy.special import kl_div
from sklearn.metrics import mean_squared_error
import tqdm

from utils import (
    amino_acid_codes, alphabet, cdn_spectrum_to_matrix, 
    nuc_spectrum_to_matrix, collect_possible_changes,
    get_equilibrium_probabilities, plot_exp_heatmap,
    prepare_aa_subst, plot_aa_eq_freqs, prepare_exp_aa_subst
)

In [57]:
# load some spectrum
clades_spectra = pd.read_csv('data/rates_by_clade.csv')
clades_spectra['Mut'] = clades_spectra['mut_type'].str.replace('to', '>')
clades_spectra20A = clades_spectra[clades_spectra.clade == '20A'].copy()
clades_spectra20A['rate'] /= clades_spectra20A['rate'].sum()
clades_spectra20A.sort_values('rate')

,clade,mut_type,count,total_count,fraction,parent_nt,parent_nt_frac,rate,Mut
11,20A,TtoG,395,17202,0.022962,T,0.508360,0.006130,T>G
4,20A,CtoG,130,17202,0.007557,C,0.137040,0.007484,C>G
0,20A,AtoC,286,17202,0.016626,A,0.289620,0.007791,A>C
9,20A,TtoA,565,17202,0.032845,T,0.508360,0.008769,T>A
2,20A,AtoT,515,17202,0.029938,A,0.289620,0.014029,A>T
7,20A,GtoC,137,17202,0.007964,G,0.064987,0.016632,G>C
3,20A,CtoA,339,17202,0.019707,C,0.137040,0.019518,C>A
10,20A,TtoC,2598,17202,0.151030,T,0.508360,0.040320,T>C
1,20A,AtoG,1617,17202,0.094001,A,0.289620,0.044050,A>G
6,20A,GtoA,837,17202,0.048657,G,0.064987,0.101614,G>A


In [ ]:
# calculate equilibrium for codon freqs without stopcodons removal

gc=1
coda = CodonAnnotation(gc)
df_changes = collect_possible_changes(gc)
spectrum_dict = clades_spectra20A.set_index('Mut')['rate'].to_dict()

df_changes['rate'] = df_changes['sbs'].map(spectrum_dict)

cdn_sbs = df_changes.groupby(['cdn1', 'cdn2'])['rate'].sum()
M = cdn_spectrum_to_matrix(cdn_sbs)
eq_prob = get_equilibrium_probabilities(M).astype(float)

eq_freqs_cdn = pd.Series(dict(zip(possible_codons, eq_prob)))
eq_freqs_cdn.name = 'eq_freq'
eq_freqs_cdn.index.name = 'cdn'
eq_freqs_cdn = eq_freqs_cdn.reset_index()
eq_freqs_cdn['aa'] = eq_freqs_cdn['cdn']\
    .map(coda.translate_codon).map(amino_acid_codes)

eq_freqs_aa = eq_freqs_cdn[eq_freqs_cdn.aa !='*'].groupby('aa')['eq_freq'].sum()
eq_freqs_aa /= eq_freqs_aa.sum()
eq_freqs_aa = eq_freqs_aa.sort_values(ascending=False).reset_index()

/tmp/ipykernel_132055/2632793336.py:11: ComplexWarning: Casting complex values to real discards the imaginary part
  eq_prob = get_equilibrium_probabilities(M).astype(float)


In [ ]:
# calculate equilibrium for codon freqs WITH stopcodons removal

def _get_equilibrium_probabilities(M):
    evals, evecs = np.linalg.eig(M)
    # find zero eigenvalue
    # ii = np.argmin(np.abs(evals))
    for vali in np.argsort(np.abs(evals)):
        val = evals[vali]
        # print(vali, val)
        if val != 0:
            ii = vali
            break
    # print(np.argsort(np.abs(evals)), ii)
    assert np.abs(evals[ii])<1e-10
    # pull out corresponding eigenvector, return normalized to sum_i p_i = 1
    p = evecs[:,ii]
    return p/p.sum()

cdn_sbs2 = df_changes.query('aa1 != "*" & aa2 != "*"').groupby(['cdn1', 'cdn2'])['rate'].sum()
M2 = cdn_spectrum_to_matrix(cdn_sbs2)
eq_prob2 = _get_equilibrium_probabilities(M2).astype(float)

eq_freqs_cdn2 = pd.Series(dict(zip(possible_codons, eq_prob2)))
eq_freqs_cdn2.name = 'eq_freq2'
eq_freqs_cdn2.index.name = 'cdn'
eq_freqs_cdn2 = eq_freqs_cdn2.reset_index()
eq_freqs_cdn2['aa'] = eq_freqs_cdn2['cdn']\
    .map(coda.translate_codon).map(amino_acid_codes)

eq_freqs_aa2 = eq_freqs_cdn2[eq_freqs_cdn2.aa !='*'].groupby('aa')['eq_freq2'].sum()
eq_freqs_aa2 /= eq_freqs_aa2.sum()
eq_freqs_aa2 = eq_freqs_aa2.sort_values(ascending=False).reset_index()

/tmp/ipykernel_132055/2700778489.py:21: ComplexWarning: Casting complex values to real discards the imaginary part
  eq_prob2 = _get_equilibrium_probabilities(M2).astype(float)


In [ ]:
# Look, stopcodons removal don't bias the Eq freqs
# (we see changes of freqs only on low abundant AA)

d = eq_freqs_aa.merge(eq_freqs_aa2)
d['mape, %'] = ((d['eq_freq'] - d['eq_freq2']) / d['eq_freq']).round(4) * 100
d['weight, %'] = (d['eq_freq'] / d['eq_freq'].sum()).round(3)*100
d

,aa,eq_freq,eq_freq2,"mape, %","weight, %"
0,Phe,0.453163,0.453574,-0.09,45.3
1,Leu,0.160812,0.161001,-0.12,16.1
2,Ile,0.119522,0.119107,0.35,12.0
3,Tyr,0.099586,0.099776,-0.19,10.0
4,Ser,0.058378,0.058466,-0.15,5.8
5,Asn,0.021885,0.021673,0.97,2.2
6,Val,0.020914,0.020946,-0.15,2.1
7,Cys,0.016945,0.016979,-0.20,1.7
8,Thr,0.012011,0.011964,0.39,1.2
9,His,0.009731,0.009741,-0.10,1.0


In [96]:
# Let's try to run eq freqs estimation on AA level without stopcodons consideration

def aa_spectrum_to_matrix(aa_sbs):
    '''
    convert dictionary of mutation counts to mutation matrix
    '''
    states = list(amino_acid_codes.values())[:-1] # without stopcodons 
    n = len(states)
    M = np.zeros((n, n))
    for i1,aa1 in enumerate(states):
        for i2,aa2 in enumerate(states):
            # if aa1!=aa2:
            if (aa1, aa2) in aa_sbs.index:
                M[i2,i1] = aa_sbs.loc[(aa1, aa2)]
    # normalize off-diagonal rates (just for standardization, doesn't affect the results)
    M /= M.sum()
    # fill the diagonal with 'outflow' term to guarantee conservation of probability
    d = M.sum(axis=0)
    np.fill_diagonal(M,-d)
    return M, states

aa_sbs = df_changes.groupby(['aa1', 'aa2'])['rate'].sum()
mat, states = aa_spectrum_to_matrix(aa_sbs)

In [97]:
aa_sbs

aa1  aa2
*    *      0.291329
     Arg    0.049089
     Cys    0.021820
     Gln    0.080641
     Glu    0.012261
              ...   
Val  Ile    0.304843
     Leu    0.714824
     Met    0.101614
     Phe    0.648296
     Val    1.000000
Name: rate, Length: 189, dtype: float64

In [103]:
def _get_equilibrium_probabilities(M):
    evals, evecs = np.linalg.eig(M)
    # find zero eigenvalue
    # ii = np.argmin(np.abs(evals))
    for vali in np.argsort(np.abs(evals)):
        val = evals[vali]
        # print(vali, val)
        if val != 0:
            ii = vali
            break
    print(evals)
    assert np.abs(evals[ii])<1e-2, evals[ii]
    # pull out corresponding eigenvector, return normalized to sum_i p_i = 1
    p = evecs[:,ii]
    return p/p.sum()

In [104]:
# we see too high eigen values, that probably cannot lead us to equilibrium
new_freqs = pd.Series(_get_equilibrium_probabilities(mat).astype(float), index=states).reset_index()
new_freqs.columns=['aa', 'new_aa_eq_freq']

[-0.12913313+0.j         -0.10968901+0.j         -0.11101737+0.j
 -0.1009315 +0.j         -0.0947494 +0.j         -0.07484093+0.j
 -0.06899938+0.j         -0.06302078+0.j         -0.03609046+0.j
 -0.03538677+0.j         -0.03333907+0.j         -0.00976595+0.j
 -0.02142484+0.00020974j -0.02142484-0.00020974j -0.01181071+0.j
 -0.01827239+0.j         -0.01345669+0.j         -0.01474923+0.j
 -0.01644793+0.j         -0.01544962+0.j        ]


/tmp/ipykernel_132055/2516446740.py:2: ComplexWarning: Casting complex values to real discards the imaginary part
  new_freqs = pd.Series(_get_equilibrium_probabilities(mat).astype(float), index=states).reset_index()


In [ ]:
# some strange result (if we run using rates from amino acid to itself (aa1->aa1) 
# some estimated freqs will be negative...)

d = eq_freqs_aa.merge(new_freqs)
d['mape, %'] = abs((d['eq_freq'] - d['new_aa_eq_freq']) / d['eq_freq']).round(4) * 100
d

,aa,eq_freq,new_aa_eq_freq,"mape, %"
0,Phe,0.453163,0.234887,48.17
1,Leu,0.160812,0.021843,86.42
2,Ile,0.119522,0.245335,105.26
3,Tyr,0.099586,0.103294,3.72
4,Ser,0.058378,0.009754,83.29
5,Asn,0.021885,0.069898,219.39
6,Val,0.020914,0.018974,9.28
7,Cys,0.016945,0.015953,5.85
8,Thr,0.012011,0.015540,29.38
9,His,0.009731,0.012757,31.09
